In [258]:
import telebot
from telebot import types
import requests
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(content_types=['text'])
def start(message):
    if message.text == '/start':
        bot.send_message(message.from_user.id, 'Я задам несколько вопросов, если на все ответ положительный, ответьте "да"\n '+\
                        '1. Вы являетесь гражданином РФ?\n ' +\
                        '2. Имеете постоянную или временную регистрацию в регионе присутствия кредитующего подразделения банка?\n ' +\
                        '3. Вам от 18  до 75 лет?\n ' +\
                        '4. Ваш официальный доход 10000 рублей или более, и вы можете это подтвердить?\n ' +\
                        '5. В настоящий момент не имеете задолженности какому-либо банку?\n ');
        bot.register_next_step_handler(message, general_q); 
    else:
        bot.send_message(message.from_user.id,
                         'Здравствуйте, это чат-бот _банка для оформления заявки на ' +\
                             'потребительский кредит \nНапишите /start, чтобы начать');
        
def general_q(message):
    if message.text.lower() == 'да':
        bot.send_message(message.from_user.id, 'Ответьте, пожалуйста, на несколько вопросов\n ' +\
                        '1. Являетесь ли вы клиентом нашего банка?\n' +\
                        '2. На какую сумму и на какой срок хотите взять кредит?\n ' +\
                        '3. Какое у вас образование?\n ' +\
                        '4. Количество детей?\n ' +\
                        '5. Ваше семейное положение?\n ' +\
                        '6. Есть ли у вас недвижимость, перечислите?\n')
        
        bot.register_next_step_handler(message, get_job)
    else:
        bot.send_message(message.from_user.id, 'К сожалению, мы пока не можем с вами сотрудничать')

def get_job(message):
    global gen_q
    gen_q = message.text
    bot.send_message(message.from_user.id, 'Теперь несколько вопросов про вашу работу\n' +\
                        'и после проверки оператора перейдем к оформлению :)\n'
                        '1. Полное название организации\n' +\
                         #предполгаю, что кто-то проверит эти данные, если
                         #клиент подходит, спрашиваем уже паспортные данные
                        '2. Ваша должность\n ' +\
                        '3. ИНН работодателя\n ' +\
                        '4. Адрес работы\n ' +\
                        '5. Рабочий телефон\n ' +\
                        '6. Когда вы начали работать в компании\n ' +\
                        '7. Заработная плата в месяц\n')
    bot.register_next_step_handler(message, get_yes_no)

def get_yes_no(message):
    global job
    job = message.text
    bot.send_message(message.from_user.id, 'Верно?\nВаша фамилия: '+str(message.from_user.last_name)+'\nВаше имя: '+str(message.from_user.first_name)+'')
    bot.register_next_step_handler(message, yes_no)

def yes_no(message):
    
    if message.text.lower() == 'да':
        global name
        global surname
        name = str(message.from_user.first_name)
        surname = str(message.from_user.last_name)
        bot.send_message(message.from_user.id, 'Напишите свое отчество')
        bot.register_next_step_handler(message, get_patronymic)
    elif message.text.lower() == 'нет':    
        bot.send_message(message.from_user.id, 'Напишите свою фамилию')
        bot.register_next_step_handler(message, get_surname)
    
def get_surname(message):
    global surname
    surname = message.text
    bot.send_message(message.from_user.id, 'Напишите свое имя')
    bot.register_next_step_handler(message, get_name)
    
def get_name(message):
    global name
    name = message.text
    bot.send_message(message.from_user.id, 'Напишите свое отчество')
    bot.register_next_step_handler(message, get_patronymic)

def get_patronymic(message):
    global patronymic
    patronymic = message.text
    bot.send_message(message.from_user.id, 'Напишите свой контактный телефон')
    bot.register_next_step_handler(message, get_phone_num)
    
def get_phone_num(message):
    global phone_num
    phone_num = message.text
    bot.send_message(message.from_user.id, 'Напишите дату рождения в формате дд.мм.гггг')
    bot.register_next_step_handler(message, get_birthdate)    
    
def get_birthdate(message):
    global birthdate
    birthdate = message.text
    if not is_date(birthdate):
        bot.send_message(message.from_user.id, 'Неправильный формат даты, попробуйте ещё раз.')
        bot.register_next_step_handler(message, get_birthdate) 
    else:
        bot.send_message(message.from_user.id, 'Напишите серию и номер паспорта в формате хх хх хххххх')
        bot.register_next_step_handler(message, get_docnumber)
    
def get_docnumber(message):
    global docnumber
    docnumber = message.text
    bot.send_message(message.from_user.id, 'Напишите дату выдачи паспорта в формате дд.мм.гггг')
    bot.register_next_step_handler(message, get_docdate)

    
def get_docdate(message):
    global docdate
    docdate = message.text
    if not is_date(docdate):
        bot.send_message(message.from_user.id, 'Неправильный формат даты, попробуйте ещё раз')
        bot.register_next_step_handler(message, get_docdate)
    else:
        bot.send_message(message.from_user.id, 'Все ли верно?\n '
                        '1. Ваша фамилия: '+surname+'\n ' +\
                        '2. Ваше имя: '+name+'\n ' +\
                        '3. Ваше отчество: '+patronymic+'\n ' +\
                        '4. Ваш телефон: '+phone_num+'\n' +\
                        '5. Ваша дата рождения: '+birthdate+'\n ' +\
                        '6. Серия и номер паспорта: '+docnumber+'\n ' +\
                        '7. Дата выдачи паспорта: '+docdate+'\n '
                        '8. Ваш ИНН: '+get_inn()+' ')
    bot.register_next_step_handler(message, get_check)
    
def get_check(message):
    if message.text.lower() == 'да':
        bot.send_message(message.from_user.id, 'Ожидайте, мы проверяем вашу заявку')
    else:
        bot.send_message(message.from_user.id, 'Попробуйте заново /start')

def get_inn():
    def suggest_inn(surname_, name_, patronymic_, birthdate_, doctype_, docnumber_, docdate_):
        url = "https://service.nalog.ru/inn-proc.do"
        data ={
            "fam": surname_,
            "nam": name_,
            "otch": patronymic_,
            "bdate": birthdate_,
            "doctype": doctype_,
            "docno": docnumber_,
            "docdt": docdate_,
            "c": "innMy",
            "captcha": "",
            "captchaToken": ""
        }
        
        resp = requests.post(url=url, data=data)
        resp.raise_for_status()
        return resp.json()['inn'] 

    inn = suggest_inn(
        surname_= surname,
        name_= name,
        patronymic_= patronymic,
        birthdate_= birthdate,
        doctype_=DocumentType.passport_russia.value,
        docnumber_= docnumber,
        docdate_= docdate)
    
    return (inn)


def is_date(date):
    if len(date) == 10 and date[2] == '.' and date[5] == '.' and int(s[0:2]) <= 31 and int(s[3:5]) <= 12 and int(s[6:10]) > 1900:
        return True
    else:
        return False

bot.polling(none_stop=True)